In [1]:
# Rode o código se precisar instalar o snscrape
#%pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Rode esse código se precisar instalar o Pandas
# !pip install pandas

In [1]:
# Imports
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt


In [2]:
def ScrapTweets(max_tweets: int, search_string: str):
    # Definindo o máximo de tweets
    maxTweets = max_tweets

    # Criando a lista para salvar os dados
    tweets_list = []

    # Usando o TwitterSearchScraper para fazer o scrape dos dados data e salvar em uma lista
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(search_string).get_items()
    ):
        if i > maxTweets:
            break
        tweets_list.append(
            [
                tweet.id,
                tweet.date,
                tweet.rawContent,
                tweet.lang,
            ]
        )
    # Criando um dataframe com os dados
    tweets_df = pd.DataFrame(
        tweets_list,
        columns=[
            "TweetId",
            "DataHora",
            "Texto",
            "Idioma",
        ],
    )
    # Adicionando a coluna de data, mês e hora
    tweets_df["dia"] = tweets_df.DataHora.dt.day
    tweets_df["mes"] = tweets_df.DataHora.dt.month
    tweets_df["ano"] = tweets_df.DataHora.dt.year

    return tweets_df


In [38]:
# Na string de busca você pode usar ou omitir os marcadores from e until
# Exemplo:  'rockinrio from:anitta until:2020-07-31'

tweets_triste = ScrapTweets(50000, "#triste")
tweets_triste["emocao"] = "triste"

tweets_chateado = ScrapTweets(25000, "#chateado")
tweets_chateado["emocao"] = "chateado"

tweets_chateada = ScrapTweets(25000, "#chateada")
tweets_chateada["emocao"] = "chateado"

tweets_feliz = ScrapTweets(50000, "#feliz")
tweets_feliz["emocao"] = "feliz"

tweets_amor = ScrapTweets(50000, "#amor")
tweets_amor["emocao"] = "amor"

tweets_raiva = ScrapTweets(50000, "#raiva")
tweets_raiva["emocao"] = "raiva"

tweets_inveja = ScrapTweets(50000, "#inveja")
tweets_inveja["emocao"] = "inveja"

tweets_ironia = ScrapTweets(50000, "#ironia")
tweets_ironia["emocao"] = "ironia"

tweets = pd.concat(
    [
        tweets_triste,
        tweets_chateado,
        tweets_chateada,
        tweets_feliz,
        tweets_amor,
        tweets_raiva,
        tweets_inveja,
        tweets_ironia,
    ]
)

# Exportar dataframe para CSV
tweets.to_csv("text-query-tweets.csv", sep=",", index=False)
